In [63]:
import json
import random
import utils

import datasets
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq


def load_from_dialogsum(file_path):
    ''' load dialoguesum jsonl data '''
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    data_dict = {}
    for sample in data:
        data_dict[sample['fname']] = {
            'summary1': sample['summary1'],
            'topic1': sample['topic1'],
            'summary2': sample['summary2'],
            'topic2': sample['topic2'],
            'summary3': sample['summary3'],
            'topic3': sample['topic3']}

    return data_dict

In [65]:
dialogsum_test = load_from_dialogsum("./data/dialogsum/dialogsum.test.jsonl")
print(dialogsum_test['test_0'])

{'summary1': 'Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.', 'topic1': 'communication method', 'summary2': 'In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.', 'topic2': 'company policy', 'summary3': 'Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office. They argue about its reasonability but #Person1# still insists.', 'topic3': 'dictation'}


In [94]:
def load_from_result(file_path):
    ''' load result json data '''
    data = []
    
    with open(file_path, 'r') as f:
        for line in f:
            data.extend(json.loads(line))
    # print(data)
    data_dict = {}
    for sample in data:
        try:
                data_dict[sample['fname']] = {
                'gen_summary1': sample['gen_summary1'],
                'gen_summary2': sample['gen_summary2'],
                'gen_summary3': sample['gen_summary3']
            }
        except:
            pass
    return data_dict

In [95]:
result = load_from_result("./result/length-topic.json")
print(result['test_0'])

{'gen_summary1': '#Person1# and #Person2# talk about the presidential election. They both think that Trump is not the right person for the job.', 'gen_summary2': '#Person1# cannot imagine if Trump were to be their President again, while #Person2# has nothing but faith in Trump.', 'gen_summary3': '#Person1# asks Ms. Dawson to take a dictation for him and asks her to send an intra-office memorandum to all employees by this afternoon. The memo says the use of Instant Message programs by employees during working hours is strictly prohibited and applies to internal and external communications.'}


In [105]:
def show_all_result(test_id, num):
    topic_no = "topic" + str(num)
    summary_no = "summary" + str(num)
    dialogsum_test = load_from_dialogsum("./data/dialogsum/dialogsum.test.jsonl")
    print("Topic: ",dialogsum_test[test_id][topic_no])
    print('*'*50 + summary_no + '*'*50)
    print(dialogsum_test[test_id][summary_no])
    list_method = ['baseline', 'topic', 'length', 'topic-length', 'length-topic']
    for i in list_method:
        print('-'*50 + i + '-'*50)
        path = f"./result/{i}.json"
        result = load_from_result(path)
        print(result[test_id]['gen_' + summary_no])

In [106]:
show_all_result('test_0', 1)

Topic:  communication method
**************************************************summary1**************************************************
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.
--------------------------------------------------baseline--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells her that Instant Message programs are strictly prohibited in the office. He also says the policy applies to internal and external communications and any employee who persists in using Instant Messaging will be placed on probation and face termination.
--------------------------------------------------topic--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and asks her to send an intra-office memorandum to all employees by this afternoon. The memo says the use of Inst

In [107]:
show_all_result('test_0', 2)

Topic:  company policy
**************************************************summary2**************************************************
In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.
--------------------------------------------------baseline--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells her that Instant Message programs are strictly prohibited in the office. He also says the policy applies to internal and external communications and any employee who persists in using Instant Messaging will be placed on probation and face termination.
--------------------------------------------------topic--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation of the company policy and asks her to send it out as an intra-office memorandum to 

In [108]:
show_all_result('test_0', 3)

Topic:  dictation
**************************************************summary3**************************************************
Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office. They argue about its reasonability but #Person1# still insists.
--------------------------------------------------baseline--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells her that Instant Message programs are strictly prohibited in the office. He also says the policy applies to internal and external communications and any employee who persists in using Instant Messaging will be placed on probation and face termination.
--------------------------------------------------topic--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and asks her to send an intra-office memorandum to all employees by this afternoon. The memo says the use of Ins

In [109]:
show_all_result('test_1', 1)

Topic:  public transportation
**************************************************summary1**************************************************
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use public transportations to keep healthy and to protect the environment.
--------------------------------------------------baseline--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it would be better for the environment and give him more freedom.
--------------------------------------------------topic--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it's less stressful and it's better for the environment.
--------------------------------------------------length--------------------------------------------------
#Person2# got stuck

In [110]:
show_all_result('test_1', 2)

Topic:  transportation
**************************************************summary2**************************************************
#Person2# decides to follow #Person1#'s suggestions on quitting driving to work and will try to use public transportations.
--------------------------------------------------baseline--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it would be better for the environment and give him more freedom.
--------------------------------------------------topic--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it's less stressful and it's better for the environment.
--------------------------------------------------length--------------------------------------------------
#Person2# got stuck in traffic again and #Person1# sugges

In [111]:
show_all_result('test_1', 3)

Topic:  discuss transportation
**************************************************summary3**************************************************
#Person2# complains to #Person1# about the traffic jam, #Person1# suggests quitting driving and taking public transportation instead.
--------------------------------------------------baseline--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it would be better for the environment and give him more freedom.
--------------------------------------------------topic--------------------------------------------------
#Person2# got stuck in traffic again. #Person1# suggests #Person2 # start taking public transport system to work because it's less stressful and it's better for the environment.
--------------------------------------------------length--------------------------------------------------
#Person2# got stuck in traffic again. #